# Лабораторная работа по теме: "Сбор и подготовка данных с помощью языка Python"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import sqlalchemy as sql

Mounted at /content/drive


In [ ]:
pip install odfpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160672 sha256=69f33b119aecc0ea2f42acd68cf910d53683ae266886ba8775eff06bbf1c494b
  Stored in directory: /root/.cache/pip/wheels/c8/2e/95/90d94fe33903786937f3b8c33dd88807f792359c6424b40469
Successfully built odfpy


### Задание №1

Для заданий этого занятия требуется загрузить датасет 'Employee_Salary_Dataset.ods' ([страница датасета на Kaggle](https://www.kaggle.com/datasets/anninasimon/employee-salary-dataset)). Датасет можно загрузить из директории datasets. Нужные данные находятся на листе 'Sheet1'. В датасете содержатся срез данных о зарплатах различных сотрудников. Загрузите эти данные в виде датафрейма.

In [ ]:
df_salary = pd.read_csv('/content/drive/MyDrive/Employee_Salary_Dataset.csv')
df_salary.head(3)

,ID,Experience_Years,Age,Gender,Salary
0,1,5,28,Female,250000
1,2,1,21,Male,50000
2,3,3,23,Female,170000


### Задание №2

Выведите общую информацию об этом датафрейме.



In [ ]:
df_salary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                35 non-null     int64 
 1   Experience_Years  35 non-null     int64 
 2   Age               35 non-null     int64 
 3   Gender            35 non-null     object
 4   Salary            35 non-null     int64 
dtypes: int64(4), object(1)
memory usage: 1.5+ KB


### Задание №3

Приведите все названия столбцов к нижнему регистру.

In [ ]:
df_salary.columns = [column.lower() for column in df_salary.columns.values]
df_salary.head(3)

,id,experience_years,age,gender,salary
0,1,5,28,Female,250000
1,2,1,21,Male,50000
2,3,3,23,Female,170000


### Задание №4*

Удалите выбросы по зарплате (salary) при помощи IQR3 (тройной межквартильный интервал). Сохраните в отдельном датафрейме срез без выбросов.

In [ ]:
Q1 = df_salary['salary'].quantile(0.25)
Q3 = df_salary['salary'].quantile(0.75)
IQR = Q3 - Q1
l_bound = Q1 - (3 * IQR)
u_bound = Q3 + (3 * IQR)
df_salary_old = df_salary
df_salary = df_salary[(df_salary['salary'] >= l_bound) & (df_salary['salary'] <= u_bound)]
df_salary.head(3)

,id,experience_years,age,gender,salary
0,1,5,28,Female,250000
1,2,1,21,Male,50000
2,3,3,23,Female,170000


### Задание №5

Из страницы в википедии https://ru.wikipedia.org/wiki/%D0%94%D0%BE%D1%85%D0%BE%D0%B4%D1%8B_%D0%BD%D0%B0%D1%81%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8 получите таблицу с данными о среднемесячных доходах в субъектах Сибирского ФО.

In [ ]:
df_regions_income = pd.read_html(
    'https://ru.wikipedia.org/wiki/%D0%94%D0%BE%D1%85%D0%BE%D0%B4%D1%8B_%D0%BD%D0%B0%D1%81%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8',
)[11]
df_regions_income.head(3)

Субъект                      2015                    2016        
        Субъект         Субъект.1 октябрь  ноябрь декабрь  январь февраль
0  Сибирский ФО      Сибирский ФО  29 500  29 400  36 500  28 700  28 500
1           NaN  Республика Алтай  22 000  21 800  29 000  21 900  22 500
2           NaN           Бурятия  29 000  28 700  36 500  28 100  27 800

### Задание №6

Запишите загруженный датафрейм с доходами в виде таблицы incomes внутри файла sqlite в директории с блокнотом. Не сохраняйте индекс, в случае, если название таблицы уже есть в БД - перепишите эту таблицу.

In [ ]:
engine = sql.create_engine('sqlite:////regions_income.db')
with engine.connect() as conn:
  df_regions_income.to_sql('incomes', conn, if_exists='replace', index=False)

In [ ]:
md = sql.MetaData()
table = sql.Table('incomes', md, autoload_with=engine)
with engine.connect() as conn:
  incomes_data = conn.execute(table.select()).fetchall()
pd.DataFrame(incomes_data).head(3)

,"('Субъект', 'Субъект')","('Субъект', 'Субъект.1')","('2015', 'октябрь')","('2015', 'ноябрь')","('2015', 'декабрь')","('2016', 'январь')","('2016', 'февраль')"
0,Сибирский ФО,Сибирский ФО,29 500,29 400,36 500,28 700,28 500
1,None,Республика Алтай,22 000,21 800,29 000,21 900,22 500
2,None,Бурятия,29 000,28 700,36 500,28 100,27 800
